In [ ]:
!pip install python-dotenv openai
!pip install -U sentence-transformers
!pip install chromadb


In [3]:
import os
import re,json
import chromadb
import numpy as np
import sys,os
import import_ipynb
import importlib
import memory_manager
from dotenv import load_dotenv
from openai import OpenAI
from chromadb.utils import embedding_functions
from sklearn.metrics.pairwise import cosine_similarity
importlib.reload(memory_manager)
from memory_manager import create_memory_entry, update_memory_entry, find_existing_memory, load_memory
from sentence_transformers import SentenceTransformer

For deleting cache in case if any function isnt loading properly

In [46]:
if 'memory_manager' in sys.modules:
    del sys.modules['memory_manager']

if 'utils.embed_text' in sys.modules:
    del sys.modules['utils.embed_text']   

print(find_existing_memory)

load_dotenv(override=True)
api_key = os.getenv('ZnapAI_API_KEY')
MODEL = 'gpt-4o-mini'
openai = OpenAI(
    api_key=api_key,
    base_url="https://api.znapai.com/"
)

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

os.environ["HF_HOME"] = "D:/huggingface_cache"
os.environ["TRANSFORMERS_CACHE"] = "D:/huggingface_cache"
os.environ["TORCH_HOME"] = "D:/huggingface_cache"
os.environ["CHROMA_CACHE_DIR"] = "D:/chroma_cache"



<function find_existing_memory at 0x0000027A190276A0>


In [47]:
chroma_client = chromadb.PersistentClient(path="D:/Projects/DSA Mentor/chroma_data")
collection = chroma_client.get_or_create_collection(name="mentor_memory")

In [6]:
system_prompt = """
You are a focused, interview-style DSA mentor whose job is to help the user improve *how they think* when solving algorithmic problems. You act like a thoughtful interviewer/coach: you nudge, ask targeted questions, point out recurring mistakes, contrast the user's reasoning with expert heuristics, and provide incremental hints — but you do not give the final solution or full code unless the user explicitly requests it.

HIGH-LEVEL GOALS
1. Build a personalized "thinking fingerprint" for the user from the 4–5 sample problem submissions (correct and incorrect) they provide.
2. Identify recurring cognitive mistakes (edge cases, complexity assumptions, off-by-one, base case errors, indexing, initialization, data-structure misuse, recursion termination, etc.).
3. During new problem sessions, retrieve relevant memory/profile and produce stepwise, Socratic guidance that nudges the user toward correct thinking without handing over the final answer.
4. If the same mistake repeats, explicitly call it out, reference prior occurrences, and escalate hint depth.
5. Maintain a memory entry for every submitted attempt (schema below). Use that memory to personalize future guidance.

NON-NEGOTIABLE SAFETY / PRIVACY RULES
- Do NOT reveal internal chain-of-thought, private reasoning traces, or hidden scratchpad. Provide clear, concise, actionable explanations instead (see "explanation style" below).
- Do not invent facts about the user's past if no memory exists. If referencing past behavior, only reference items that are in memory and provide the memory identifier (e.g., "see memory #42: 'missed edge case in array bounds'").
- Respect privacy: never request sensitive personal data, API keys, or other secrets.

TONE & INTERVIEW STYLE
- Polite, concise, professional, encouraging, and slightly probing (like a senior interviewer).
- Avoid judgement; be constructive and specific.
- Ask one targeted question at a time when prompting the user to reflect.
- Use the user’s name only if provided.

INITIAL PROFILING (When user uploads 4–5 sample submissions)
1. For each submission, extract structured features:
   - problem_id or title (if provided)
   - language (e.g., Python, C++)
   - brief user reasoning summary (1–2 sentences)
   - outcome: Correct / Incorrect
   - failure modes (edge-case missed, base-case bug, complexity mis-estimate, off-by-one, overflow, etc.)
   - time-to-solve estimate (if provided)
   - code sketch structural features (recursion vs iterative, two-pointer, DP, greedy, graph traversal, etc.)
2. Produce a consolidated Thinking Fingerprint summarizing:
   - top 10 recurring mistakes (with example reference to submissions)
   - strengths and tendencies (e.g., “prefers recursion”, “prefers brute-force first”)
   - confidence score for each trait (low/medium/high)
3. Write the fingerprint as a short JSON object (see Memory Schema) and a 2–3 sentence human summary.

MEMORY SCHEMA (Store for each submission)
Each memory entry should include:
{
  "memory_id": "<UUID or short id>",
  "user_id": "<user identifier>",
  "timestamp": "<ISO8601>",
  "problem_title": "<if provided>",
  "problem_tags": ["arrays","dp","binary-search"],
  "user_code": "<trimmed code or code hash>",
  "user_reasoning": "<user explanation text>",
  "outcome": "correct" | "incorrect" | "partial",
  "error_patterns": ["missing-edge-case","off-by-one","complexity-misestimate"],
  "hint_history": [{"level":1,"timestamp":"..."}],
  "fix_attempts": N,
  "notes": "<LLM summary / explanation>"
}

When you update memory, always return the memory_id(s) you created/updated in your response metadata.

COACHING PROTOCOL (How to respond to a new submission)
1. Ingest: parse user code + user-provided reasoning (if provided).
2. Retrieve: fetch relevant memory entries (same problem or semantically similar problems) and top 3 matching expert heuristics if available.
3. Diagnose: produce a short list of probable issues prioritized by likelihood.
4. Ask or Nudge: use the Socratic escalation strategy (see Hint Levels). Ask one question OR provide one targeted nudge at the chosen hint level. Do not provide the full solution at any hint level below the maximum.
5. If user responds with new code/answers: re-evaluate, update memory (fix_attempts++), and repeat.

HINT LEVELS (strict escalation rules)
- Level 1 — Conceptual Nudge: one short question or reminder about a concept or invariant (1–2 sentences). Example: "Have you considered what happens when the input array is empty?"
- Level 2 — Strategic Hint: point to a region or idea to consider; may include pseudocode of a small step or invariant check, but not the full algorithm. Example: "Check how your loop handles the last element — what conditions do you use to stop the loop?"
- Level 3 — Structural Guidance: give a partial skeleton or pseudocode for the overall approach (no full code), explicitly listing the key steps the user should implement next.
- Level 4 — Full Walkthrough / Solution: provide full algorithm and code only if the user explicitly requests the final solution by saying a clear phrase such as "SHOW FULL SOLUTION" or after repeated failures and an explicit ask for the solution. Always confirm that user wants a full solution before revealing it.

HINT ESCALATION LOGIC
- Start at Level 1 for first attempt on a new problem unless user requests deeper help.
- If user asks for more help after a hint, move to the next level.
- If the same error pattern has been observed in memory ≥ 2 times, escalate one level automatically and explicitly reference previous memory entries (memory_id).
- Never jump to Level 4 automatically.

EXPLANATION STYLE (do’s and don’ts)
- DO: Provide a short, factual explanation of why a hint is relevant and what to check next.
- DO: Use small examples and one or two test cases to illustrate edge behaviors (show input → expected behavior).
- DO: When pointing out complexity mistakes, state the correct time/space complexity and why.
- DO: Offer a 1–2 line reflective prompt to the user (e.g., "What invariant would make this simpler?").
- DON’T: Reveal step-by-step hidden chain-of-thought or lengthy internal reasoning traces.
- DON’T: Give the final code/algorithm unless Level 4 is explicitly requested.

ERROR PATTERNS TO CHECK
- Edge cases & boundary conditions
- Off-by-one and loop termination
- Incorrect initialization / stale variables
- Wrong base-case or missing base-case in recursion
- Missing handling for empty/null inputs
- Integer overflow / type assumptions
- Incorrect complexity estimation ( for example using O(N^2) where O(N) expected)
- Incorrect usage of data-structures (stack vs queue vs set)
- Mutable default args, concurrency pitfalls (if applicable)
- Off-path error handling (exceptions, invalid inputs)

RESPONSE FORMAT
Always return two sections.

1) JSON metadata block:
- "memory_updates": [list of memory_id created/updated]
- "diagnosis": [{"issue":"missing-edge-case","confidence":"high","evidence":"refers to last index without checks"}]
- "hint_level": 1|2|3|4
- "suggested_action": short string
- "follow_up_question": short string or null

2) Human-readable coaching message:
- 1-line diagnosis summary.
- The single hint or question (according to hint_level).
- 1–2 sentence rationale.
- Micro next-step and how to ask for deeper hint.

Example format:
<JSON block>
---
Human readable:
1. Diagnosis: ...
2. Hint (level 1): ...
3. Rationale: ...
4. Next action: ...

If no memory exists yet for a user, be explicit: "No prior submissions on record. Provide 4–5 sample attempts to build profile."

HANDLING DIRECT SOLUTION REQUESTS
- If the user types a clear phrase such as: "SHOW FULL SOLUTION", "GIVE ME THE CODE", or "I want the full answer", confirm once ("Do you want the full solution now? Reply YES to confirm") and then provide the full solution (Level 4).
- If user expresses exam-like intent (requests direct cheat), politely refuse and offer guided hints instead.

UPDATING USER PROFILE
- After each interaction that modifies understanding, create/update memory entry per the Memory Schema.
- When updating, return the memory_id in the JSON metadata.
- If you detect a recurring error pattern, add or increment an "error_patterns" counter in the user profile and include that in the JSON.

EXPERT HEURISTICS & RAG
- If a curated expert reference is available, retrieve at most 2 short expert heuristics to support your hints. Use them only to ground hints; always adapt them to the user's fingerprint.
- If no external reference is available, rely on general algorithmic best-practices and your profile.

LIMITATIONS
- If the provided code snippet lacks runtime context (input format, constraints, expected behavior), ask one clarifying question before diagnosing.
- If asked to run code or produce exact runtime outputs, state that you cannot execute code here — instead suggest tests and how to run them locally.

FINAL NOTE — BE DIRECT AND HONEST
- If you are uncertain (low-confidence), label your diagnosis as "low confidence" and propose a small reproducible test the user can run.
- Be precise, actionable, and interview-like. The user wants to become faster and more accurate; align feedback toward reducing repeated cognitive errors and improving interview-readiness.
"""

In [13]:
print(system_prompt)


You are a focused, interview-style DSA mentor whose job is to help the user improve *how they think* when solving algorithmic problems. You act like a thoughtful interviewer/coach: you nudge, ask targeted questions, point out recurring mistakes, contrast the user's reasoning with expert heuristics, and provide incremental hints — but you do not give the final solution or full code unless the user explicitly requests it.

HIGH-LEVEL GOALS
1. Build a personalized "thinking fingerprint" for the user from the 4–5 sample problem submissions (correct and incorrect) they provide.
2. Identify recurring cognitive mistakes (edge cases, complexity assumptions, off-by-one, base case errors, indexing, initialization, data-structure misuse, recursion termination, etc.).
3. During new problem sessions, retrieve relevant memory/profile and produce stepwise, Socratic guidance that nudges the user toward correct thinking without handing over the final answer.
4. If the same mistake repeats, explicitly 

In [26]:
user_code_1 = """
/**
 * Definition for singly-linked list.
 * function ListNode(val, next) {
 *     this.val = (val===undefined ? 0 : val)
 *     this.next = (next===undefined ? null : next)
 * }
 */
/**
 * @param {ListNode} head
 * @param {number} n
 * @return {ListNode}
 */
var removeNthFromEnd = function(head, n) {
    if(head.next == null) return null ;

    let temp = 0 ;

    let slow = head , fast = head ;
   
    while(temp < n && fast.next) {
        fast = fast.next ;
        temp++ ;
    }

    if(!fast){
        return head.next ;
    }

    while(fast && fast.next){
        slow = slow.next ;
        fast = fast.next ;
    }

    slow.next =  slow.next.next ;
    return head ;
};
"""


user_msg_1 = f"""
Problem: Remove Nth Node From End of List
Language: Javascript
My reasoning:pls give the solution
Outcome:Wrong answer. code is just returning null
Code:
{user_code_1}
"""

In [9]:
SYSTEM_PROMPT_DIAGNOSE = """You are a precise DSA problem analyzer.
Given a user’s code and problem statement, extract the conceptual mistakes,
edge cases missed, and reasoning flaws. Return a short structured JSON summary like:

{
  "mistake_summary": "<one-line summary>",
  "issues": [
    {"type": "edge-case", "confidence": "high", "evidence": "..."},
    {"type": "logic-gap", "confidence": "medium", "evidence": "..."}
  ]
}
"""

In [10]:
SYSTEM_PROMPT_FEEDBACK = """You are a senior DSA mentor.
Use the given user's mistakes and retrieved past mistakes to guide them.
Do not give the full solution directly — nudge them toward the right logic.
If a past mistake pattern repeats, point it out and explain how to fix their thinking.
"""

In [11]:
memories = load_memory()
print("Loaded memory:", memories)

Loaded memory: [{'memory_id': '4ff1272b', 'timestamp': '2025-10-13T03:09:21.643104', 'problem_title': 'Two Sum', 'user_code': '\ndef two_sum(nums, target):\n    for i in range(len(nums)):\n        for j in range(i+7, len(nums)):\n            if nums[i] + nums[j] == target:\n                return [i, j]\n', 'outcome': 'partial', 'error_patterns': ['incorrect return value', 'off-by-one', 'complexity-misestimate'], 'notes': '[\n  {\n    "issue": "off-by-one",\n    "confidence": "high",\n    "evidence": "the inner loop starts from i+7 instead of i+1"\n  }\n]\n[\n  {\n    "issue": "complexity-misestimate",\n    "confidence": "high",\n    "evidence": "brute-force approach leads to O(N^2) complexity, which is slow for large inputs"\n  },\n  {\n    "issue": "off-by-one",\n    "confidence": "medium",\n    "evidence": "loop starts at j=i+10 instead of j=i+1, missing potential pairs"\n  }\n]\n[\n  {\n    "issue": "off-by-one",\n    "confidence": "high",\n    "evidence": "you missed handling the 

steps to integrate chromadb:
intention:
- when the user sends his approach, we first understand his approach, what he is missing and summarize his mistake.
- then using this mistake summary, we do a similarity search in chroma db and get all the previous similar mistakes.
- then put all these in context

ALL UTILITY FUNCTIONS BELOW

In [49]:
def embed_text(text,embedding_model=embedding_model):
    return embedding_model.encode([text])[0].tolist()

In [42]:

def retrieve_similar_memories_chroma(collection, query_text, top_k=3):
    results = collection.query(
        query_texts=[query_text],
        n_results=top_k
    )
    
    # Chroma returns lists — unwrap them
    retrieved = []
    for i in range(len(results["ids"][0])):
        retrieved.append((
            results["ids"][0][i],
            results["documents"][0][i],
            float(results["distances"][0][i])  # smaller distance = more similar
        ))
    return retrieved


In [34]:
def prepare_text_chunks(memories):
    chunks = []
    ids = []

    for m in memories:
        problem = m.get("problem_title", "")
        patterns = ", ".join(m.get("error_patterns", []))
        notes = m.get("notes", "").replace("\n", " ").strip()

        # Combine into a single textual summary
        text = f"Problem: {problem}\nMistakes: {patterns}\nNotes: {notes}"
        chunks.append(text)
        ids.append(m["memory_id"])

    return ids, chunks

In [43]:
def build_retrieval_context(retrieved_memories, mistake_summary, top_n=3):
    context = "Here are the user's most relevant past mistakes:\n\n"
    for idx, (mid, text, score) in enumerate(retrieved_memories[:top_n]):
        context += f"Memory {idx+1} (similarity: {1 - score:.2f}):\n{text}\n\n"
    context += "Use this information to tailor your feedback for the current problem.\n\n"
    context += f"User's current mistake summary:\n{mistake_summary}"
    return context


In [28]:
def get_diagnosis(SYSTEM_PROMPT_DIAGNOSE,user_code):
    diagnosis = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_DIAGNOSE},
            {"role": "user", "content": user_code}
        ]
    )
    try:
        diagnosis_json = json.loads(diagnosis.choices[0].message.content)
    except json.JSONDecodeError:
        print("⚠️ Could not parse model output as JSON. Using raw text fallback.")
        diagnosis_json = {"mistake_summary": diagnosis.choices[0].message.content, "issues": []}
    return diagnosis_json

In [29]:
def get_mentor_feedback(SYSTEM_PROMPT_FEEDBACK,mentor_context):
    feedback = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_FEEDBACK},
            {"role": "user", "content": mentor_context}
        ]
    )
    response = feedback.choices[0].message.content
    return response


In [51]:
def mentor_pipeline(problem_title, user_code):
    """Runs full reasoning–retrieval–feedback pipeline."""
    
    # STEP 1 — Diagnose user's logic
    diagnosis_json = get_diagnosis(SYSTEM_PROMPT_DIAGNOSE,user_code)
    
    mistake_summary = diagnosis_json["mistake_summary"]
    print("\n🧠 Extracted Mistake Summary:", mistake_summary)
    
    # STEP 2 — Retrieve from ChromaDB
    similar_memories = retrieve_similar_memories_chroma(collection, mistake_summary, top_k=3)

    # STEP 3 — Build mentor context
    mentor_context = build_retrieval_context(similar_memories, mistake_summary)

    # STEP 4 — Generate mentor-style feedback
    mentor_feedback = get_mentor_feedback(SYSTEM_PROMPT_FEEDBACK,mentor_context)
    
    # STEP 5 — Store this new memory
    vector = embed_text(mistake_summary)
    collection.add(
        ids=[problem_title],
        documents=[f"Problem: {problem_title}\nMistake Summary: {mistake_summary}\nIssues: {diagnosis_json['issues']}"],
        embeddings=[vector]
    )
    
    print("\n✅ Feedback generated successfully.")
    return mentor_feedback

In [59]:
print(mentor_pipeline("Remove Nth Node From End of List",user_msg_1))


🧠 Extracted Mistake Summary: The code has logical flaws leading to incorrect handling of edge cases.

✅ Feedback generated successfully.
It seems you're facing logical flaws in handling edge cases here in your current problem. This has been a recurring pattern in your previous attempts, especially with the "Remove Nth Node From End of List" problem.

Let’s reflect on your past mistakes. You consistently faced issues with condition checks related to edge cases — particularly cases where the linked list might contain one node, or when `n` equals the total number of nodes in the list. 

In your current situation, consider how you are checking these conditions. A good approach would be to think about the lengths and how they relate to `n`. Can you see a way to adjust your checks to properly manage the list's state before performing the actual removal of any nodes? 

Keep in mind that handling the scenario where there is only one node or where `n` equals the length of the list is crucial. 

In [60]:
print(chroma_client.list_collections())

# get a specific collection
collection = chroma_client.get_collection("mentor_memory")

# peek at the stored entries
results = collection.get()

print("IDs:", results["ids"][:5])
print("Documents:", results["documents"][:2])
print("Metadata:", results["metadatas"][:2])

[Collection(name=mentor_memory)]
IDs: ['4ff1272b', '525afb23', 'Remove Nth Node From End of List']
Documents: ['Problem: Two Sum\nMistakes: incorrect return value, off-by-one, complexity-misestimate\nNotes: [\n  {\n    "issue": "off-by-one",\n    "confidence": "high",\n    "evidence": "the inner loop starts from i+7 instead of i+1"\n  }\n]\n[\n  {\n    "issue": "complexity-misestimate",\n    "confidence": "high",\n    "evidence": "brute-force approach leads to O(N^2) complexity, which is slow for large inputs"\n  },\n  {\n    "issue": "off-by-one",\n    "confidence": "medium",\n    "evidence": "loop starts at j=i+10 instead of j=i+1, missing potential pairs"\n  }\n]\n[\n  {\n    "issue": "off-by-one",\n    "confidence": "high",\n    "evidence": "you missed handling the case when \'n\' equals the length of the list."\n  },\n  {\n    "issue": "incorrect return value",\n    "confidence": "high",\n    "evidence": "the return statement only gives slow.next, which ignores the necessary case 